In [20]:
import pandas as pd
import helpers.processing_helpers as ph

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer

from sklearn.neural_network import  MLPRegressor

from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline


In [33]:
df_dev = pd.read_csv("./dataset/development.csv")

In [34]:
noise_indexes = [0,7,12,15,16,17]
acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

sensors_removed = df_dev.drop(columns=ph.get_column_names(features, noise_indexes))
df = sensors_removed.drop(columns=ph.get_column_names(['tmax', 'rms', 'area'], acc_idxs))

In [35]:
score = make_scorer(ph.mean_euclid_dist, greater_is_better=False)

In [36]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', MLPRegressor(random_state=42, max_iter=200, n_iter_no_change=200, learning_rate_init=0.01))
])

In [37]:
y_train = df[['x', 'y']].copy()

X_train = df.drop(columns=['x', 'y'])

In [38]:
param_grid = {'clf__hidden_layer_sizes' : [(50,),
                                            (25, 25),
                                            (20, 10, 20),
                                            (15, 10, 10, 15)]
              }

In [39]:
gridsearch = GridSearchCV(pipe, param_grid, scoring=score, cv=3, verbose=2)
gridsearch.fit(X_train, y_train)

c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 4 candidates, totalling 12 fits


c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Arist\OneDrive\Skrivebord\DSL labs\DSL_project2\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

In [28]:
results = pd.concat([pd.DataFrame(gridsearch.cv_results_["params"]),
                     pd.DataFrame(gridsearch.cv_results_["mean_test_score"], columns=["MED"]),
                     pd.DataFrame(gridsearch.cv_results_["mean_fit_time"], columns=["Time"])],
                     axis=1)
df = results.sort_values('MED', ascending=False)

# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
 
# All dataframes hereafter reflect these changes.
display(df)

pd.reset_option('all')

,clf__hidden_layer_sizes,MED,Time
2,"(20, 10, 20)",-61.419441,26.189165
1,"(25, 25)",-84.327397,32.616908
3,"(15, 10, 10, 15)",-87.633545,34.569028
0,"(50,)",-120.817792,19.870611


C:\Users\Arist\AppData\Local\Temp\ipykernel_20084\3228783991.py:16: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')
